In [2]:
import pandas as pd

In [3]:
import datacompy

In [4]:
from excel_maker import Support

ModuleNotFoundError: No module named 'payout_app'

In [5]:
from postgress_config import PostgressDB

In [6]:
import time

In [12]:
from oracle_config import Ora

In [8]:
import numpy as np

In [8]:
sup = Support()

In [9]:
main_dataframe = pd.read_excel(r'resources\main_dataframe.xlsx')

In [10]:
main_dataframe

,SERVICE_DATE,PATIENT_ID,EPISODE_ID,PATIENT_NAME,SERVICE_CODE,SERVICE_DESC,PATIENT_TYPE,GROSS_AMOUNT,NET_AMOUNT,DOCTOR_SHARE,DOCTOR_NAME,PHYSICIAN_ID,SOURCE OF DATA,DOCTORS_GROUP,REFERENCE_GROUP,COMMENTS,REV_STREAM
0,02/08/2022,KH1000714877,13965578,Mr. Kewal Gandhi,CNOP000010,ER Consultation,Emergency,1500.0,1500.0,1500.0,DR SANTOSH WAIGANKAR,51006979,IP Admitting Doc wise rev rep,D,OP Consultation,NaN,1.0
1,03/08/2022,KH1000714877,13965578,Mr. Kewal Gandhi,CNIP000046,Doctor Routine Visit - Surgical Oncology,General,1000.0,945.0,945.0,DR SANTOSH WAIGANKAR,51006979,IP Admitting Doc wise rev rep,D,IP Consultation,NaN,1.0
2,04/08/2022,KH1000714877,13965578,Mr. Kewal Gandhi,CNIP000046,Doctor Routine Visit - Surgical Oncology,General,1000.0,945.0,945.0,DR SANTOSH WAIGANKAR,51006979,IP Admitting Doc wise rev rep,D,IP Consultation,NaN,1.0
3,05/08/2022,KH1000714877,13965578,Mr. Kewal Gandhi,CNIP000046,Doctor Routine Visit - Surgical Oncology,General,1000.0,945.0,945.0,DR SANTOSH WAIGANKAR,51006979,IP Admitting Doc wise rev rep,D,IP Consultation,NaN,1.0
4,06/08/2022,KH1000714877,13965578,Mr. Kewal Gandhi,CNIP000046,Doctor Routine Visit - Surgical Oncology,General,1000.0,945.0,945.0,DR SANTOSH WAIGANKAR,51006979,IP Admitting Doc wise rev rep,D,IP Consultation,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48170,2022-08-27 12:37:24,KH1000768304,139910560001,NaN,CARD,NaN,EHC,400.0,400.0,400.0,Dr.PRAVIN KAHALE,NaN,EHC Query,C,OP Consultation,NaN,1.0
48171,2022-08-29 14:44:23,KH1000146958,139923420001,NaN,CARD,NaN,EHC,400.0,400.0,400.0,Dr.SUNIL WANI,NaN,EHC Query,C,OP Consultation,NaN,1.0
48172,2022-08-30 11:25:34,KH1000639737,139936900001,NaN,CARD,NaN,EHC,400.0,400.0,400.0,Dr.PRAVIN KAHALE,NaN,EHC Query,C,OP Consultation,NaN,1.0
48173,2022-08-30 11:27:43,KH1000072397,139936890001,NaN,CARD,NaN,EHC,400.0,400.0,400.0,Dr.PRAVIN KAHALE,NaN,EHC Query,C,OP Consultation,NaN,1.0


In [10]:
#df = pd.read_excel(r'resources\doc.xlsx')
#d = datacompy.Compare(main_dataframe,df,join_columns='PATIENT_NAME')
#sup.excel_generator(excel_data=d.df1_unq_rows, page_name="master")

In [11]:
#main_dataframe = sup.negative_to_postive_entry_check(main_dataframe)

In [1]:
main_dataframe = pd.read_excel(r'resources\main_dataframe.xlsx')
main_dataframe1 = main_dataframe.copy()

NameError: name 'pd' is not defined

In [13]:
def negative_to_postive_entry_check(dataframe):

    negative_share_rows = dataframe[ dataframe['DOCTOR_SHARE'] < 0]

    
    for _, row in negative_share_rows.iterrows():
        doctor_share = abs(row['DOCTOR_SHARE'])
        doctor_name = row['DOCTOR_NAME']
        episode_id = row['EPISODE_ID']
        comments = str(dataframe.loc[row.name, "COMMENTS"])+ "   ---   " if str(dataframe.loc[row.name, "COMMENTS"]) != 'nan' else ""  # Convert float to string


        matching_rows = dataframe[
            (dataframe['DOCTOR_SHARE'] == doctor_share) &
            (dataframe['DOCTOR_NAME'] == doctor_name) &
            (dataframe['EPISODE_ID'] == episode_id )
        ]
        if not matching_rows.empty:
            dataframe.loc[row.name, "COMMENTS"] = comments + " Positive Entry Found"
        else:
            dataframe.loc[row.name, "COMMENTS"] = comments + " No!!! Positive Entry Found. Manual Check is Required"


   
    return dataframe

In [14]:
main_dataframe1 = negative_to_postive_entry_check(main_dataframe1)

ValueError: Can only compare identically-labeled Series objects

In [19]:
def plastic_cosmetic_gst(dataframe):
    sql_qurey ="""
    select PRACTITIONER_NAME,LONG_DESC
            from AM_PRACTITIONER,AM_SPECIALITY
            where PRIMARY_SPECIALITY_CODE =  speciality_code
            and LONG_DESC in ('PLASTIC SURGERY','DERMATOLOGY','DENTAL SURGERY','AESTHETIC CLINIC')"""
    db = Ora()
    dr_list, _ = db.run_query(sql_qurey)
    # dr_list, _ = db.gst_on_cosmetic_and_plastic()
    episode_id = np.array(
        list(
            map(
                str,
                dataframe[dataframe["DOCTOR_NAME"].isin(dr[0] for dr in dr_list)][
                    "EPISODE_ID"
                ].unique(),
            )
        )
    ).astype(int)

    df_filtered = dataframe[dataframe["EPISODE_ID"].isin(episode_id)]
    sql_qurey = """
    SELECT RULE_CODE,blng_serv_code
            FROM BL_ADDL_CHARGE_RULE_BY_CS
            WHERE BLNG_SERV_CODE IN (SELECT blng_Serv_code from bl_patient_charges_interface 
            where patient_id= {variable} 
            and episode_id= {variable1})
    """
    db = Ora()
    tax_data = np.array(
        np.unique(
            [
                data_tuple[1]
                for data, _ in [
                    db.run_query_without_self_close_db(sql_qurey.format(variable=f"'{patient_id}'",variable1=f"'{episode_id}'"))
                    for patient_id, episode_id in zip(
                        df_filtered["PATIENT_ID"].tolist(),
                        df_filtered["EPISODE_ID"].tolist(),
                    )
                ]
                if data
                for data_tuple in data
            ]
        )
    )
    db.close_connection()
    all_entery_of_episode = np.unique(
        dataframe[dataframe["SERVICE_CODE"].isin(tax_data)]["EPISODE_ID"].to_numpy()
    )
    dataframe.loc[
        dataframe["EPISODE_ID"].isin(all_entery_of_episode), "COMMENTS"
    ] = (
        dataframe.loc[
            dataframe["EPISODE_ID"].isin(all_entery_of_episode), "COMMENTS"
        ].fillna("")
        + " Changed from "
        + dataframe.loc[
            dataframe["EPISODE_ID"].isin(all_entery_of_episode), "DOCTOR_SHARE"
        ].astype(str)
    )
    dataframe.loc[
        dataframe["EPISODE_ID"].isin(all_entery_of_episode), "DOCTOR_SHARE"
    ] = np.divide(
        dataframe.loc[
            dataframe["EPISODE_ID"].isin(all_entery_of_episode), "DOCTOR_SHARE"
        ],
        1.18,
    )

In [20]:
plastic_cosmetic_gst(main_dataframe)


    select PRACTITIONER_NAME,LONG_DESC
            from AM_PRACTITIONER,AM_SPECIALITY
            where PRIMARY_SPECIALITY_CODE =  speciality_code
            and LONG_DESC in ('PLASTIC SURGERY','DERMATOLOGY','DENTAL SURGERY','AESTHETIC CLINIC')

    SELECT RULE_CODE,blng_serv_code
            FROM BL_ADDL_CHARGE_RULE_BY_CS
            WHERE BLNG_SERV_CODE IN (SELECT blng_Serv_code from bl_patient_charges_interface 
            where patient_id= 'KH1000807762' 
            and episode_id= '13838456')
    

    SELECT RULE_CODE,blng_serv_code
            FROM BL_ADDL_CHARGE_RULE_BY_CS
            WHERE BLNG_SERV_CODE IN (SELECT blng_Serv_code from bl_patient_charges_interface 
            where patient_id= 'KH1000807762' 
            and episode_id= '13838456')
    

    SELECT RULE_CODE,blng_serv_code
            FROM BL_ADDL_CHARGE_RULE_BY_CS
            WHERE BLNG_SERV_CODE IN (SELECT blng_Serv_code from bl_patient_charges_interface 
            where patient_id= 'KH1000807762' 
   


    SELECT RULE_CODE,blng_serv_code
            FROM BL_ADDL_CHARGE_RULE_BY_CS
            WHERE BLNG_SERV_CODE IN (SELECT blng_Serv_code from bl_patient_charges_interface 
            where patient_id= 'KH1000031907' 
            and episode_id= '13963006')
    

    SELECT RULE_CODE,blng_serv_code
            FROM BL_ADDL_CHARGE_RULE_BY_CS
            WHERE BLNG_SERV_CODE IN (SELECT blng_Serv_code from bl_patient_charges_interface 
            where patient_id= 'KH1000031907' 
            and episode_id= '13963006')
    

    SELECT RULE_CODE,blng_serv_code
            FROM BL_ADDL_CHARGE_RULE_BY_CS
            WHERE BLNG_SERV_CODE IN (SELECT blng_Serv_code from bl_patient_charges_interface 
            where patient_id= 'KH1000031907' 
            and episode_id= '13963006')
    

    SELECT RULE_CODE,blng_serv_code
            FROM BL_ADDL_CHARGE_RULE_BY_CS
            WHERE BLNG_SERV_CODE IN (SELECT blng_Serv_code from bl_patient_charges_interface 
            where patient_id= '


    SELECT RULE_CODE,blng_serv_code
            FROM BL_ADDL_CHARGE_RULE_BY_CS
            WHERE BLNG_SERV_CODE IN (SELECT blng_Serv_code from bl_patient_charges_interface 
            where patient_id= 'KH1000821324' 
            and episode_id= '13959535')
    

    SELECT RULE_CODE,blng_serv_code
            FROM BL_ADDL_CHARGE_RULE_BY_CS
            WHERE BLNG_SERV_CODE IN (SELECT blng_Serv_code from bl_patient_charges_interface 
            where patient_id= 'KH1000821324' 
            and episode_id= '13959535')
    

    SELECT RULE_CODE,blng_serv_code
            FROM BL_ADDL_CHARGE_RULE_BY_CS
            WHERE BLNG_SERV_CODE IN (SELECT blng_Serv_code from bl_patient_charges_interface 
            where patient_id= 'KH1000821324' 
            and episode_id= '13959535')
    

    SELECT RULE_CODE,blng_serv_code
            FROM BL_ADDL_CHARGE_RULE_BY_CS
            WHERE BLNG_SERV_CODE IN (SELECT blng_Serv_code from bl_patient_charges_interface 
            where patient_id= '


    SELECT RULE_CODE,blng_serv_code
            FROM BL_ADDL_CHARGE_RULE_BY_CS
            WHERE BLNG_SERV_CODE IN (SELECT blng_Serv_code from bl_patient_charges_interface 
            where patient_id= 'KH1000267555' 
            and episode_id= '13969450')
    

    SELECT RULE_CODE,blng_serv_code
            FROM BL_ADDL_CHARGE_RULE_BY_CS
            WHERE BLNG_SERV_CODE IN (SELECT blng_Serv_code from bl_patient_charges_interface 
            where patient_id= 'KH1000132649' 
            and episode_id= '13970692')
    

    SELECT RULE_CODE,blng_serv_code
            FROM BL_ADDL_CHARGE_RULE_BY_CS
            WHERE BLNG_SERV_CODE IN (SELECT blng_Serv_code from bl_patient_charges_interface 
            where patient_id= 'KH1000630008' 
            and episode_id= '13970944')
    

    SELECT RULE_CODE,blng_serv_code
            FROM BL_ADDL_CHARGE_RULE_BY_CS
            WHERE BLNG_SERV_CODE IN (SELECT blng_Serv_code from bl_patient_charges_interface 
            where patient_id= '


    SELECT RULE_CODE,blng_serv_code
            FROM BL_ADDL_CHARGE_RULE_BY_CS
            WHERE BLNG_SERV_CODE IN (SELECT blng_Serv_code from bl_patient_charges_interface 
            where patient_id= 'KH1000830070' 
            and episode_id= '13965945')
    

    SELECT RULE_CODE,blng_serv_code
            FROM BL_ADDL_CHARGE_RULE_BY_CS
            WHERE BLNG_SERV_CODE IN (SELECT blng_Serv_code from bl_patient_charges_interface 
            where patient_id= 'KH1000830070' 
            and episode_id= '13965945')
    

    SELECT RULE_CODE,blng_serv_code
            FROM BL_ADDL_CHARGE_RULE_BY_CS
            WHERE BLNG_SERV_CODE IN (SELECT blng_Serv_code from bl_patient_charges_interface 
            where patient_id= 'KH1000830070' 
            and episode_id= '13965945')
    

    SELECT RULE_CODE,blng_serv_code
            FROM BL_ADDL_CHARGE_RULE_BY_CS
            WHERE BLNG_SERV_CODE IN (SELECT blng_Serv_code from bl_patient_charges_interface 
            where patient_id= '

...


    SELECT RULE_CODE,blng_serv_code
            FROM BL_ADDL_CHARGE_RULE_BY_CS
            WHERE BLNG_SERV_CODE IN (SELECT blng_Serv_code from bl_patient_charges_interface 
            where patient_id= 'KH1000826503' 
            and episode_id= '13945251')
    

    SELECT RULE_CODE,blng_serv_code
            FROM BL_ADDL_CHARGE_RULE_BY_CS
            WHERE BLNG_SERV_CODE IN (SELECT blng_Serv_code from bl_patient_charges_interface 
            where patient_id= 'KH1000826503' 
            and episode_id= '13987927')
    

    SELECT RULE_CODE,blng_serv_code
            FROM BL_ADDL_CHARGE_RULE_BY_CS
            WHERE BLNG_SERV_CODE IN (SELECT blng_Serv_code from bl_patient_charges_interface 
            where patient_id= 'KH1000333604' 
            and episode_id= '13989195')
    

    SELECT RULE_CODE,blng_serv_code
            FROM BL_ADDL_CHARGE_RULE_BY_CS
            WHERE BLNG_SERV_CODE IN (SELECT blng_Serv_code from bl_patient_charges_interface 
            where patient_id= '


    SELECT RULE_CODE,blng_serv_code
            FROM BL_ADDL_CHARGE_RULE_BY_CS
            WHERE BLNG_SERV_CODE IN (SELECT blng_Serv_code from bl_patient_charges_interface 
            where patient_id= 'KH1000189581' 
            and episode_id= '13977306')
    

    SELECT RULE_CODE,blng_serv_code
            FROM BL_ADDL_CHARGE_RULE_BY_CS
            WHERE BLNG_SERV_CODE IN (SELECT blng_Serv_code from bl_patient_charges_interface 
            where patient_id= 'KH1000696580' 
            and episode_id= '13992009')
    

    SELECT RULE_CODE,blng_serv_code
            FROM BL_ADDL_CHARGE_RULE_BY_CS
            WHERE BLNG_SERV_CODE IN (SELECT blng_Serv_code from bl_patient_charges_interface 
            where patient_id= 'KH1000756983' 
            and episode_id= '13980390')
    

    SELECT RULE_CODE,blng_serv_code
            FROM BL_ADDL_CHARGE_RULE_BY_CS
            WHERE BLNG_SERV_CODE IN (SELECT blng_Serv_code from bl_patient_charges_interface 
            where patient_id= '


    SELECT RULE_CODE,blng_serv_code
            FROM BL_ADDL_CHARGE_RULE_BY_CS
            WHERE BLNG_SERV_CODE IN (SELECT blng_Serv_code from bl_patient_charges_interface 
            where patient_id= 'KH1000832946' 
            and episode_id= '13982733')
    

    SELECT RULE_CODE,blng_serv_code
            FROM BL_ADDL_CHARGE_RULE_BY_CS
            WHERE BLNG_SERV_CODE IN (SELECT blng_Serv_code from bl_patient_charges_interface 
            where patient_id= 'KH1000828579' 
            and episode_id= '13977527')
    

    SELECT RULE_CODE,blng_serv_code
            FROM BL_ADDL_CHARGE_RULE_BY_CS
            WHERE BLNG_SERV_CODE IN (SELECT blng_Serv_code from bl_patient_charges_interface 
            where patient_id= 'KH1000764759' 
            and episode_id= '13992123')
    

    SELECT RULE_CODE,blng_serv_code
            FROM BL_ADDL_CHARGE_RULE_BY_CS
            WHERE BLNG_SERV_CODE IN (SELECT blng_Serv_code from bl_patient_charges_interface 
            where patient_id= '


    SELECT RULE_CODE,blng_serv_code
            FROM BL_ADDL_CHARGE_RULE_BY_CS
            WHERE BLNG_SERV_CODE IN (SELECT blng_Serv_code from bl_patient_charges_interface 
            where patient_id= 'KH1000769201' 
            and episode_id= '13976076')
    

    SELECT RULE_CODE,blng_serv_code
            FROM BL_ADDL_CHARGE_RULE_BY_CS
            WHERE BLNG_SERV_CODE IN (SELECT blng_Serv_code from bl_patient_charges_interface 
            where patient_id= 'KH1000788036' 
            and episode_id= '13983676')
    

    SELECT RULE_CODE,blng_serv_code
            FROM BL_ADDL_CHARGE_RULE_BY_CS
            WHERE BLNG_SERV_CODE IN (SELECT blng_Serv_code from bl_patient_charges_interface 
            where patient_id= 'KH1000821768' 
            and episode_id= '13994275')
    

    SELECT RULE_CODE,blng_serv_code
            FROM BL_ADDL_CHARGE_RULE_BY_CS
            WHERE BLNG_SERV_CODE IN (SELECT blng_Serv_code from bl_patient_charges_interface 
            where patient_id= '


    SELECT RULE_CODE,blng_serv_code
            FROM BL_ADDL_CHARGE_RULE_BY_CS
            WHERE BLNG_SERV_CODE IN (SELECT blng_Serv_code from bl_patient_charges_interface 
            where patient_id= 'KH1000825139' 
            and episode_id= '13978634')
    

    SELECT RULE_CODE,blng_serv_code
            FROM BL_ADDL_CHARGE_RULE_BY_CS
            WHERE BLNG_SERV_CODE IN (SELECT blng_Serv_code from bl_patient_charges_interface 
            where patient_id= 'KH1000299698' 
            and episode_id= '13965454')
    

    SELECT RULE_CODE,blng_serv_code
            FROM BL_ADDL_CHARGE_RULE_BY_CS
            WHERE BLNG_SERV_CODE IN (SELECT blng_Serv_code from bl_patient_charges_interface 
            where patient_id= 'KH1000834364' 
            and episode_id= '13990498')
    

    SELECT RULE_CODE,blng_serv_code
            FROM BL_ADDL_CHARGE_RULE_BY_CS
            WHERE BLNG_SERV_CODE IN (SELECT blng_Serv_code from bl_patient_charges_interface 
            where patient_id= '

In [ ]:
sup.excel_generator(excel_data=main_dataframe1, page_name="main_dataframe1")